In [2]:
from src.util_classes import get_train_test_dataloaders, LogMelspec, train
from src.configs import TaskConfig, StreamingTaskConfig
from src.model import CRNN, StreamingCRNN
from src.wandb_pipeline import train_baseline

import torch

baseline_task_config = TaskConfig(
    hidden_size=32,
    bottleneck_size=32
)

train_loader, val_loader = get_train_test_dataloaders("speech_commands", baseline_task_config)

baseline_melspec_train = LogMelspec(is_train=True, config=baseline_task_config)
baseline_melspec_val = LogMelspec(is_train=False, config=baseline_task_config)

In [3]:
import wandb
from src.wandb_pipeline import evaluate_model

teacher = CRNN(baseline_task_config).cuda()

_ = wandb.restore('baseline.pt', run_path="broccoliman/kws/381ohren")

teacher.load_state_dict(torch.load("baseline.pt"))

baseline_melspec_val.melspec.cpu()

evaluate_model(teacher.cpu(), val_loader, baseline_melspec_val, "cpu")

  0%|          | 0/102 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

{'area under FA/FR curve': 2.6525748998086058e-05,
 'evaluation time (s)': 1.1379876136779785,
 'memory size (MB)': 0.10112476348876953,
 'number of parameters': 25387,
 'MACs': 54911167.058823526}

In [ ]:
teacher_fp16 = torch.quantization.quantize_dynamic(teacher, dtype=torch.float16)
evaluate_model(teacher_fp16.cpu(), val_loader, baseline_melspec_val, "cpu")

  0%|          | 0/102 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

{'area under FA/FR curve': 4.7638932340094716e-05,
 'evaluation time (s)': 1.2087290287017822,
 'memory size (MB)': 0.10608959197998047,
 'number of parameters': 808,
 'MACs': 20102823.529411763}

In [ ]:
from src.configs import DistillTaskConfig
from src.wandb_pipeline import train_distillation

distill_config = DistillTaskConfig(
    hidden_size=24,
    bottleneck_size=16,
    cnn_out_channels=6,
    distill_w=.25,
    attn_distill_w=.05,
    melspec_win_length=400,
    melspec_hop_length=160,
    num_epochs=50,
    use_scheduler=True
)

student_melspec_train = LogMelspec(is_train=True, config=distill_config)
student_melspec_val = LogMelspec(is_train=False, config=distill_config)

student = train_distillation(
    teacher,
    train_loader, 
    val_loader,
    baseline_melspec_train,
    student_melspec_train,
    student_melspec_val,
    distill_config,
    log_wandb=True,
    name_wandb="triple-50-epochs"
) 

train/attn_loss,████▇▇▆▅▆▄▄▄▃▁▂▁
train/cls_loss,█▇▇▇▇▇▆▇▅▃▄▃▂▃▃▁
train/kl_loss,▇▇███▆▆▇▆▃▅▄▄▃▄▁
train/loss,█▇██▇▇▆▆▆▃▄▃▃▃▃▁
train/lr,████▇▇▇▆▆▅▅▄▃▃▂▁
train/attn_loss,54.29375
train/cls_loss,16.37372
train/kl_loss,16.4942
train/loss,23.21196
train/lr,0.0003


Number of trainable parameters: 14321
EPOCH: 0


  0%|          | 0/405 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

EPOCH: 1


...

  0%|          | 0/102 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
test: MACs,▁
test: area under FA/FR curve,▁
test: evaluation time (s),▁
test: memory size (MB),▁
test: number of parameters,▁
train/attn_loss,█▆▄▄▄▂▄▂▃▃▂▂▃▂▂▃▂▂▁▃▂▂▄▂▂▂▁▄▃▃▂▁▂▃▂▃▂▂▄▂
train/cls_loss,█▆▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁
train/kl_loss,█▆▄▄▃▁▃▁▄▃▃▂▂▃▁▄▂▄▂▂▂▁▄▂▃▁▁▄▃▄▄▁▂▄▃▃▃▂▅▃
train/loss,█▆▄▄▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▂▁▂▂▂▂▂▁▁▁▂▂▁▁▂▁
train/lr,███████▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁


In [ ]:
student_melspec_val.melspec.cpu()
evaluate_model(student.cpu(), val_loader, student_melspec_val, "cpu")

  0%|          | 0/102 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

{'area under FA/FR curve': 5.3522934255080735e-05,
 'evaluation time (s)': 1.185582160949707,
 'memory size (MB)': 0.06118488311767578,
 'number of parameters': 14321,
 'MACs': 34605284.705882356}

In [ ]:
student_fp16 = torch.quantization.quantize_dynamic(student, dtype=torch.float16)
evaluate_model(student_fp16.cpu(), val_loader, student_melspec_val, "cpu")

  0%|          | 0/102 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

{'area under FA/FR curve': 5.3516966707499614e-05,
 'evaluation time (s)': 1.3515667915344238,
 'memory size (MB)': 0.06379222869873047,
 'number of parameters': 606,
 'MACs': 15077117.647058824}